In [2]:
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import FirefoxOptions 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import time

In [3]:
url = 'https://racetozerodataexplorer.org/proceed/'

scope1_lst = []
scope2_local_lst = []
scope2_market_lst = []
name_lst = []

master_lst = [scope1_lst, scope2_local_lst, scope2_market_lst, name_lst]

options = FirefoxOptions()
options.add_argument("--headless") # comment out to see how it works

with webdriver.Firefox(options=options) as driver:
    driver.get(url)
    headers = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/thead/tr/th")
    headers = [header.text for header in headers]
    table = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/tbody/tr")
    
    for row in table:
        name = row.find_element(By.TAG_NAME, 'th')
        print(name.text)
        master_lst[3].append(name.text)
        for i, cell in enumerate(row.find_elements(By.TAG_NAME, 'td')):
            if i % 9 in [0,1,2]: # only interested in first 3 columns
                try:
                    cell.find_element(By.CLASS_NAME, 'styles_button__IS06I').click()
                    WebDriverWait(driver, 10).until( # Wait for pop-up to load
                        EC.presence_of_element_located((By.CLASS_NAME, 'styles_axisValue__ipOq1')))
                    values = driver.find_elements(By.CLASS_NAME, 'styles_axisValue__ipOq1')
                    master_lst[i].append([value.text.replace(',','') for value in values])
                    driver.find_element(By.CLASS_NAME, 'styles_close__cpVjT').click()
                except:
                    master_lst[i].append(['', '', '', ''])

37 Interactive Entertainment
A.P. Moller - Maersk
AAK AB
AB Volvo
ABB
AbbVie Inc
Accenture
Accor
adidas AG
ADIENT plc
Adobe
AECOM
AENA SME SA
AFRY AB
Agder Energi
Agilent Technologies Inc.
Ahold Delhaize
Ajinomoto Co.Inc.
Akamai Technologies Inc
Akzo Nobel NV
Alfa Laval Corporate AB
Alibaba Group Holding Limited
Amazon
Ambev S.A
Ambuja Cements
America Movil SAB de CV
American Airlines Group Inc
American Axle & MFG Holdings Inc
American Eagle Outfitters Inc.
Americanas S.A.
Amey UK plc
Analog Devices, Inc.
Apple Inc.
Applied Materials Inc.
Aramark Corporation
Arcadis
ARÇELİK A.Ş.
Ardagh Group SA
Asahi Group Holdings, Ltd.
Ashland LLC
Asics Corporation
Askul Corporation
ASOS
Assa Abloy
AstraZeneca
Atea ASA
Atlantia
Atlassian Corporation Plc
Atos SE
Aurubis AG
Autodesk, Inc.
Autoliv Inc
Autostrade per l'italia Spa
Axiata Group Berhad
Azul Sa
Babcock International Group PLC
BAE Systems
Balfour Beatty
Barratt Developments plc
Bayer AG
BCE Inc.
Becton, Dickinson and Co.
Beiersdorf AG
Bekaert

In [49]:
from itertools import combinations

In [94]:
headers = [header.replace('\n', ' ').replace(
    '4 years trend (ending in reporting year)', '') for header in headers]
years = ['2018', '2019', '2020', '2021']

col_names = [[], [], []]
for i, header in enumerate(headers[1:4]):
    for year in years:
        col_names[i].append(f'{header} {year}')
    
# Need to merge all these 
df_scope1 = pd.DataFrame(master_lst[0], index=master_lst[3], columns=col_names[0])
df_scope2_local = pd.DataFrame(master_lst[1], index=master_lst[3], columns=col_names[1])
df_scope2_market = pd.DataFrame(master_lst[2], index=master_lst[3], columns=col_names[2])

In [95]:
# Drop Interpublic Group of Companies, Inc. which is duplicated in data source
df_scope1 = df_scope1.loc[~df_scope1.index.duplicated()] 
df_scope2_local = df_scope2_local.loc[~df_scope2_local.index.duplicated()] 
df_scope2_market = df_scope2_market.loc[~df_scope2_market.index.duplicated()] 

In [100]:
df = df_scope1.merge(
    df_scope2_local, left_index=True, right_index=True).merge(
        df_scope2_market, left_index=True, right_index=True)


df = df.loc[~df.duplicated()] # Remove companies with no data whatsoever

In [106]:
df.sample(100, random_state=1234)

,Scope 1 emissions (tCO₂e) 2018,Scope 1 emissions (tCO₂e) 2019,Scope 1 emissions (tCO₂e) 2020,Scope 1 emissions (tCO₂e) 2021,Scope 2 emissions (tCO₂e) Location 2018,Scope 2 emissions (tCO₂e) Location 2019,Scope 2 emissions (tCO₂e) Location 2020,Scope 2 emissions (tCO₂e) Location 2021,Scope 2 emissions (tCO₂e) Market 2018,Scope 2 emissions (tCO₂e) Market 2019,Scope 2 emissions (tCO₂e) Market 2020,Scope 2 emissions (tCO₂e) Market 2021
Bayer AG,1510000,2030000,2010000,1930000,1480000,1770000,1750000,1560000,1400000,1680000,1570000,1240000
DNV AS,5700,25600,1500,1600,17200,13900,7400,4100,,,,
ManpowerGroup,28521,21499,13314,18608,28702,23955,14244,9076,16118,22096,13038,7844
L'Oréal,54282,49982,43134,27668,168953,160393,133578,160368,38871,32512,13572,12971
Olam International,1697091,1977401,2913188,2679203,199959,279127,188060,185219,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
Faurecia,126793,146000,131000,173000,596327,606000,532000,492567,-,766000,665000,549091
Tapestry Inc,1048,936,10941,3523,57255,59435,38878,37914,-,57008,39294,37245
Votorantim Cimentos,21898656,19215648,20421293,24877550,821934,665974,532912,771610,,,,
Suntory Beverage & Food Limited,219000,264000,246000,251000,205000,285000,240000,264000,,,,
